# Creating Vector Database

In [ ]:
# If we run this machine on GPU machine it will be faster.

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd

In [ ]:
apipath = r'H:\\My Drive\\config\\hbqa.txt'
apipath = r'/content/drive/MyDrive/config/hbqa-colab.txt'
import configparser

config = configparser.ConfigParser()
config.read(apipath)
secret_key = config['global']['OPENAI_API_KEY']
datapath = config['global']['DATA_FOLDER']
corpuspath = config['global']['CORPUS_FOLDER']
PINECONE_API_KEY = config['global']['PINECONE_API_KEY']
PINECONE_ENV = config['global']['PINECONE_ENV']

# PINECONE is Vector Database. To store the vector so that we can quickly search the vector space.
# https://app.pinecone.io
# get PINECONE_API_KEY key from app.pinecone.io
# find your PINECONE_ENVIRONMENT next to the api key in pinecone console

# Creating Embedding for Chunks

In [ ]:
# This chunk was created to use with ChatGPT Front End to create prompt for generating QA.
# At this point of time there was no clarity around chunk size, number of question, types of question, \
# as per initial investigation cost of ChatGPT API was very high. So intially this Chunk was created. \
# Unfortuntely the main file is lost. But the prompt created from those chunks still exists. So, after\
#  reverse engineering, created Chunk from the Prompt text.
df_hbqa_for_fe = pd.read_csv(datapath+"05-QA_Final_ChatGPT_FE-Created-14Sep23.csv")
print(df_hbqa_for_fe.shape)
df_hbqa_for_fe.head(4)

(256, 7)


,Context_Len,Len,Section_Id,Chunk_Id,Prompt,Chunk,QA
0,2000,2030,Book01_046,193,"Write 1 question, corresponding answer, and th...","Sauti continued, 'The Muni, having said so unt...",\nQ1: Why did the Muni wander over the earth a...
1,2000,3462,Book01_104,425,"Write 6 question, corresponding answer, and th...",In this connection there is another old histor...,Q1: Who was the wise Rishi mentioned in the te...
2,2000,1974,Book01_007,83,"Write 1 question, corresponding answer, and th...","Sauti said, 'the god of fire enraged at the cu...",Q1: What did Agni say was the consequence of a...
3,2000,1492,Book01_005,78,"Write 1 question, corresponding answer, and th...",And the Rakshasa saw the apartment in which th...,Q1: What did the Rakshasa ask the god of fire ...


In [ ]:
# There are 256 chunks

In [ ]:
#This chunk was created to use with ChatGPT API to create prompt for QA.
df_hbqa_for_api = pd.read_csv(datapath+"05-QA_Final_ChatGPT_API-Manually-Created-13Sep23.csv")
print(df_hbqa_for_api.shape)
df_hbqa_for_api.head(4)

(300, 4)


,Section_Id,Chunk_Id_Temp,Chunk,QA
0,Book01_002,1,THE MAHABHARATA ADI PARVA Section I\nOm! Havin...,\n\nQ1: Who asked Sauti about his journey?\nA1...
1,Book01_002,2,The Rishi Vyasa published this mass of knowled...,\n\nQ1: Who asked Sauti about his journey?\nA1...
2,Book01_002,3,"Vyasa executed the compilation of the Bharata,...",\nQ1: Who composed the Bharata?\nA1: The Rishi...
3,Book01_002,4,'When I heard that Yudhishthira had been follo...,\n\nQ1. Who asked Sauti about his journey?\nA1...


In [ ]:
df_hbqa_for_api.rename(columns={'Chunk_Id_Temp':'Chunk_Id'}, inplace=True)

In [ ]:
# There are 300 chunks

In [ ]:
# Both chunk files were created with different chapters of Mahabharat Book, different chunk size, they are created
# at different time therefore chunk_id in
# the both the files will be duplicate. To avoid that problem we are manually changing chunk_id
df_hbqa_for_fe['Chunk_Id'] = df_hbqa_for_fe['Chunk_Id']+10000

In [ ]:
df_hbqa_for_fe.head(4)

,Context_Len,Len,Section_Id,Chunk_Id,Prompt,Chunk,QA
0,2000,2030,Book01_046,10193,"Write 1 question, corresponding answer, and th...","Sauti continued, 'The Muni, having said so unt...",\nQ1: Why did the Muni wander over the earth a...
1,2000,3462,Book01_104,10425,"Write 6 question, corresponding answer, and th...",In this connection there is another old histor...,Q1: Who was the wise Rishi mentioned in the te...
2,2000,1974,Book01_007,10083,"Write 1 question, corresponding answer, and th...","Sauti said, 'the god of fire enraged at the cu...",Q1: What did Agni say was the consequence of a...
3,2000,1492,Book01_005,10078,"Write 1 question, corresponding answer, and th...",And the Rakshasa saw the apartment in which th...,Q1: What did the Rakshasa ask the god of fire ...


In [ ]:
df_mahabharat_chunk = pd.concat([df_hbqa_for_fe,df_hbqa_for_api])
print(df_mahabharat_chunk.shape)
df_mahabharat_chunk.sample(4)

(556, 7)


,Context_Len,Len,Section_Id,Chunk_Id,Prompt,Chunk,QA
216,2000.0,2730.0,Book01_203,10775,"Write 6 question, corresponding answer, and th...","Duryodhana said. 'Let us, O father, by means o...",Q1: Whose advice does Duryodhana seek in order...
52,2000.0,1770.0,Book01_076,10322,"Write 4 question, corresponding answer, and th...","Vaisampayana continued, 'The learned Sukra, ha...",Q1: What consequence does Sukra declare for Br...
228,2000.0,2542.0,Book01_100,10411,"Write 5 question, corresponding answer, and th...","Vaisampayana continued, 'O Bharata, when Santa...",Q1: Why did Santanu feel no inclination to gra...
35,NaN,NaN,Book01_042,36,NaN,"Section XLI\n(Astika Parva continued)\n""Sauti ...",Q1: Who is Sesha?\nA1: Sesha is the eldest bro...


In [ ]:
df_mahabharat_chunk.to_csv(datapath+'08.1-Mahabharat_Chunk.csv')

In [ ]:
df_mahabharat_chunk.shape

(556, 7)

### Load Embedding Model. It will create 384 dim vector for the sentences.

In [ ]:
!pip install -Uq sentence-transformers

In [ ]:
from sentence_transformers import SentenceTransformer

model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')

In [ ]:
# It is time taking step

In [ ]:
DEVICE='cuda'

In [ ]:
%time
Chunk_Sentences = df_mahabharat_chunk.Chunk.tolist()
Chunk_Embeddings = model.encode(Chunk_Sentences)

CPU times: user 4 µs, sys: 0 ns, total: 4 µs
Wall time: 6.91 µs


In [ ]:
import torch

In [ ]:
ChunkVectors= torch.tensor(Chunk_Embeddings, dtype=torch.float).to(DEVICE)

In [ ]:
# Flatten the tensors into 1D arrays
ChunkVec_list = ChunkVectors.tolist()

In [ ]:
df_mahabharat_chunk['ChunkVector'] = ChunkVectors.tolist()


# Save Vector to Harddisk

In [ ]:
df_mahabharat_chunk.reset_index(inplace=True)

In [ ]:
df_mahabharat_chunk.to_csv(datapath+'09.1-Mahabharat_Chunk_Vector.csv')

In [ ]:
df_mahabharat_chunk.head(3)

,index,Context_Len,Len,Section_Id,Chunk_Id,Prompt,Chunk,QA,ChunkVector
0,0,2000.0,2030.0,Book01_046,10193,"Write 1 question, corresponding answer, and th...","Sauti continued, 'The Muni, having said so unt...",\nQ1: Why did the Muni wander over the earth a...,"[-0.027269765734672546, 0.07061152160167694, 0..."
1,1,2000.0,3462.0,Book01_104,10425,"Write 6 question, corresponding answer, and th...",In this connection there is another old histor...,Q1: Who was the wise Rishi mentioned in the te...,"[-0.04464489966630936, 0.07791255414485931, -0..."
2,2,2000.0,1974.0,Book01_007,10083,"Write 1 question, corresponding answer, and th...","Sauti said, 'the god of fire enraged at the cu...",Q1: What did Agni say was the consequence of a...,"[0.00098479597363621, 0.10335040837526321, -0...."


In [ ]:
# df_mahabharat_chunk = pd.read_csv(datapath + '09.1-Mahabharat_Chunk_Vector.csv')

# Save Context/Chunk Vector into Pinecone Vector database

In [ ]:
!pip install -Uq pinecone-client

In [ ]:
# import pinecone

# pinecone.init(
# 	api_key='ddf2ef12-9108-4e80-b438-8f86a0b617c0',
# 	environment='asia-southeast1-gcp-free'
# )
# index = pinecone.Index('HBQA')

In [ ]:
import os
import pinecone

# # get api key from app.pinecone.io
# PINECONE_API_KEY = os.environ.get('PINECONE_API_KEY') or 'PINECONE_API_KEY'
# # find your environment next to the api key in pinecone console
# PINECONE_ENV = os.environ.get('PINECONE_ENVIRONMENT') or 'PINECONE_ENVIRONMENT'

pinecone.init(
    api_key=PINECONE_API_KEY,
    environment=PINECONE_ENV
)

# index_name = 'semantic-search-fast'
index_name = "hbqa" #it must be in lower letters

In [ ]:
# !pip uninstall pinecone_datasets

In [ ]:
# from pinecone_datasets import load_dataset

# dataset = load_dataset('quora_all-MiniLM-L6-bm25')
# # we drop sparse_values as they are not needed for this example
# dataset.documents.drop(['metadata'], axis=1, inplace=True)
# dataset.documents.rename(columns={'blob': 'metadata'}, inplace=True)
# # we will use 80K rows of the dataset between rows 240K -> 320K
# dataset.documents.drop(dataset.documents.index[320_000:], inplace=True)
# dataset.documents.drop(dataset.documents.index[:240_000], inplace=True)
# dataset.head()

In [ ]:
import ast
import time

In [ ]:
# only create index if it doesn't exist
if index_name not in pinecone.list_indexes():
    pinecone.create_index(
        name=index_name,
        dimension=len(ast.literal_eval(df_mahabharat_chunk['ChunkVector'][0])),
        metric='cosine'
    )
    # wait a moment for the index to be fully initialized
    time.sleep(1)

# now connect to the index
index = pinecone.Index(index_name)

In [ ]:
# ast.literal_eval(df_mahabharat_chunk['ChunkVector'][0])

In [ ]:
df_mahabharat_chunk['ChunkVector']

0      [-0.027269765734672546, 0.07061152160167694, 0...
1      [-0.04464489966630936, 0.07791255414485931, -0...
2      [0.00098479597363621, 0.10335040837526321, -0....
3      [-0.028564315289258957, 0.05742616951465607, -...
4      [-0.08478111028671265, 0.044292598962783813, -...
                             ...                        
295    [0.013013549149036407, 0.054428279399871826, -...
296    [-0.01544362772256136, 0.0036141437012702227, ...
297    [-0.027105355635285378, 0.09472988545894623, -...
298    [-0.10657359659671783, 0.09772208333015442, 0....
299    [-0.10565254092216492, 0.02106192149221897, 0....
Name: ChunkVector, Length: 556, dtype: object

In [ ]:
chunk_id = df_mahabharat_chunk['Chunk_Id'].to_list()
# chunk_vector = [ast.literal_eval(df_mahabharat_chunk['ChunkVector'][i]) for i in range(df_mahabharat_chunk.shape[0])]
chunk_vector = [df_mahabharat_chunk['ChunkVector'][i] for i in range(df_mahabharat_chunk.shape[0])]

In [ ]:
import random
import itertools

def chunks(iterable, batch_size=100):
    """A helper function to break an iterable into chunks of size batch_size."""
    it = iter(iterable)
    chunk = tuple(itertools.islice(it, batch_size))
    while chunk:
        yield chunk
        chunk = tuple(itertools.islice(it, batch_size))

# vector_dim = 128
# vector_count = 10000

# Example generator that generates many (id, vector) pairs
# example_data_generator = map(lambda i: (f'id-{i}', [random.random() for _ in range(vector_dim)]), range(vector_count))
data_generator = map(lambda i: ( 'id'+str(chunk_id[i]), chunk_vector[i]),  range(df_mahabharat_chunk.shape[0]))

# Upsert data with 100 vectors per upsert request
i=1
batchsize = 100
for ids_vectors_chunk in chunks(data_generator, batch_size=batchsize):
    index.upsert(vectors=ids_vectors_chunk)  # Assuming `index` defined elsewhere
    print("Processed ",i*batchsize, " Vectors")
    i+=1


Processed  100  Vectors
Processed  200  Vectors
Processed  300  Vectors
Processed  400  Vectors
Processed  500  Vectors
Processed  600  Vectors


# Check whether a correct document can be retrived using a Question

In [ ]:
from random import randint
i = randint(0, df_mahabharat_chunk.shape[0])
df_mahabharat_chunk.loc[i,:]

index                                                        233
Context_Len                                               2000.0
Len                                                       2469.0
Section_Id                                            Book01_076
Chunk_Id                                                   10315
Prompt         Write 5 question, corresponding answer, and th...
Chunk          Between the celestials and the Asuras, there h...
QA             Q1: Why did the gods install Vrihaspati and th...
ChunkVector    [-0.02179011143743992, 0.05217727646231651, -0...
Name: 233, dtype: object

In [ ]:
df_mahabharat_chunk.loc[i,'QA']

'Q1: Why did the gods install Vrihaspati and the Asuras install Usanas as their priests?\nA1: The gods installed Vrihaspati as their priest and the Asuras installed Usanas as their priest for conducting their sacrifices, due to their rivalry and desire for victory.\nR1: "The gods, then, from desire of victory, installed the son of Angiras (Vrihaspati) as their priest to conduct their sacrifices; while their opponents installed the learned Usanas as their priest for the same purpose. And between those two Brahmanas there are always much boastful rivalry."\n\nQ1: How did the Asuras who were slain by the gods come back to life?\nA1: The seer Sukra, through the power of his knowledge, revived the Asuras who were slain by the gods.\nR1: "Those Danavas assembled for encounter that were slain by the gods were all revived by the seer Sukra by the power of his knowledge."\n\nQ1: Why couldn\'t Vrihaspati revive the gods who were slain by the Asuras?\nA1: Vrihaspati didn\'t possess the knowledge 

In [ ]:
query = df_mahabharat_chunk.loc[i,'QA'].split('?')[0].split(":")[1]
print(query)

# create the query vector
xq = model.encode(query).tolist()

# now query
xc = index.query(xq, top_k=3, include_metadata=True)
xc

 Why did the gods install Vrihaspati and the Asuras install Usanas as their priests


{'matches': [{'id': 'id10315', 'score': 0.68428576, 'values': []},
             {'id': 'id10105', 'score': 0.529710948, 'values': []},
             {'id': 'id10425', 'score': 0.51749593, 'values': []},
             {'id': 'id10145', 'score': 0.499064147, 'values': []},
             {'id': 'id66', 'score': 0.472976238, 'values': []},
             {'id': 'id23', 'score': 0.468029976, 'values': []},
             {'id': 'id184', 'score': 0.463395983, 'values': []},
             {'id': 'id10104', 'score': 0.463198, 'values': []},
             {'id': 'id10805', 'score': 0.462174177, 'values': []},
             {'id': 'id10146', 'score': 0.455861896, 'values': []},
             {'id': 'id108', 'score': 0.452878922, 'values': []},
             {'id': 'id10506', 'score': 0.449235111, 'values': []},
             {'id': 'id10171', 'score': 0.44322142, 'values': []},
             {'id': 'id10829', 'score': 0.441985935, 'values': []},
             {'id': 'id115', 'score': 0.435897022, 'values': []}

In [ ]:
print ("predicted chunkid ", xc['matches'][0]['id'], " Actual Chunk Id:", df_mahabharat_chunk.loc[i,:])

'id10315'